# Collecting Dataframes
This notebook collects the fooof results and compiles them into a dataframe, for both the macaque and human data, separately.

In [14]:
import numpy as np
from scipy import io
import pandas as pd
import os, sys

sys.path.append('../')
import echo_utils

In [7]:
# collect FOOOF results that were produced under these parameters
win_len, p_cur = '1sec', 'psd_med'
fg_param_to_load = 'knee'

### Macaque ECoG

In [6]:
# get cortex files for electrode coordinates
chibi_ctx_file = '/Users/rdgao/Documents/data/NeuroTycho/Propofol/GridLocations/20110621KTMD_Anesthesia+and+Sleep_Chibi_Toru+Yanagawa_mat_2Dimg/ChibiMap.mat'
george_ctx_file = '/Users/rdgao/Documents/data/NeuroTycho/Propofol/GridLocations/20110112KTMD_Anesthesia+and+Sleep_George_Toru+Yanagawa_mat_2Dimg/GeorgeMap.mat'
ctx_loc = []
for ind, ctx_file in enumerate([chibi_ctx_file, george_ctx_file]):
    ctx_mat = io.loadmat(ctx_file, squeeze_me=True)
    ctx_loc.append(np.array([ctx_mat['X'], ctx_mat['Y']]).T)

In [21]:
# collection
df_savepath = '../data/df_macaque'
df_combined = pd.DataFrame()
col_names = ['patient','cond','pharm', 'session_id', 'chan', 'exp', 'knee', 'tau','log_tau','err', 'r2']

# load fooof results
result_basepath = '/Users/rdgao/Documents/code/research/field-echos/results/neurotycho/rest_anes/'
session_resultpath = np.sort([f+'/' for f in os.listdir(result_basepath) if os.path.isdir(result_basepath+f)])
session_dict = {id:ind for ind, id in enumerate(np.unique([s.split('_')[3] for s in session_resultpath]))}

for s in session_resultpath:
    fooof_folder = result_basepath + s +'/psd/'+win_len+'/fooof/'+p_cur+'/'    
    ff_list = [ff for ff in os.listdir(fooof_folder) if '.json' in ff]
    ff_file = ff_list[np.where([fg_param_to_load in f for f in ff_list])[0][0]] # load specific fooof file
    
    # return fooof fits and convert knee to tau
    fg_aps, fg_pks, fg_err, fg_r2s = echo_utils.return_fg_fits(ff_file, fooof_folder)
    if fg_aps.shape[1]==3:
        knee_freq, knee_tau = echo_utils.convert_knee_val(fg_aps[:,1],fg_aps[:,2])
        knee = fg_aps[:,1]
    else:
        knee_tau = np.zeros_like(fg_aps[:,0])
        knee = np.zeros_like(fg_aps[:,0])
    
    df_data = np.vstack((fg_aps[:,-1], knee, knee_tau, np.log10(knee_tau), fg_err, fg_r2s)).T
    s_spl = s.split('_')
    patient, cond, pharm, session_id = s_spl[7], s_spl[1], s_spl[2], session_dict[s_spl[3]]
    chan = np.arange(len(fg_err))+1
    df_cur = pd.DataFrame(np.hstack((np.repeat(np.array([patient,cond,pharm, session_id])[:,None].T, len(fg_err), axis=0), chan[:,None], df_data))
                 ,columns=col_names)
    df_cur.insert(5,'y', ctx_loc[0 if patient is 'Chibi' else 1][:,0])
    df_cur.insert(6,'z', ctx_loc[0 if patient is 'Chibi' else 1][:,1])

    df_combined = df_combined.append(df_cur, ignore_index=True)

df_combined.columns
df_combined = df_combined.astype({c:str if c in ['patient', 'cond', 'pharm'] else np.float for c in col_names})

# saveout
df_combined.to_csv(df_savepath+'.csv')

../echo_utils.py:59: RuntimeWarning: invalid value encountered in power
  knee_freq = knee**(1./exponent)
